In [1]:
from __future__ import print_function
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import glob
import nibabel as nib
from scipy.ndimage.interpolation import zoom
import pickle

from skimage.transform import rescale, resize
import torchvision
# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [2]:
data_base_path = './'
data_folder_list = ['align_img']

In [3]:
all_BVs = []
for cur_floder in data_folder_list:
    cur_folder_path = os.path.join(data_base_path,cur_floder)
    all_BVs += glob.glob(cur_folder_path+'/label*')
print(len(all_BVs))

371


In [ ]:
img_label = nib.load(all_BVs[10])
img_label = np.uint8(img_label.get_data())
print(count, np.shape(img_label))

In [4]:
all_train_data = []
count = 0
for bv_path in all_BVs:
#     img = nib.load(img_path[0])
#     img = np.float32(img.get_data())
    img_label = nib.load(bv_path)
    img_label = np.uint8(img_label.get_data())
    x, y, z = np.shape(img_label)
    print(count, x, y, z)
    
    #img2=zoom(pad_img,1/2)
    #img_label2=zoom(pad_img_label,1/2)
    
    img_label[img_label>0] = 1
    
    label_resized = zoom(img_label, (128.0/x, 64.0/y, 64.0/z))
    label_resized[label_resized>=0.5] = 1
    label_resized[label_resized<0.5] = 0
    

    print(label_resized.shape, np.unique(label_resized))
    all_train_data.append(label_resized)
    
    img_nft = nib.Nifti1Image(label_resized,np.eye(4))
    img_save_data_path = './resize_img/rotate_img{}.nii'.format(count)
    nib.save(img_nft,img_save_data_path)
    count += 1

0 106 61 57
(128, 64, 64) [0 1]
1 102 55 51
(128, 64, 64) [0 1]
2 104 54 52
(128, 64, 64) [0 1]
3 79 75 42
(128, 64, 64) [0 1]
4 85 68 41
(128, 64, 64) [0 1]
5 86 73 49


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


(128, 64, 64) [0 1]
6 108 59 54
(128, 64, 64) [0 1]
7 92 56 52
(128, 64, 64) [0 1]
8 107 49 50
(128, 64, 64) [0 1]
9 110 54 53
(128, 64, 64) [0 1]
10 98 88 56
(128, 64, 64) [0 1]
11 100 64 46
(128, 64, 64) [0 1]
12 103 64 53
(128, 64, 64) [0 1]
13 85 62 39
(128, 64, 64) [0 1]
14 79 73 38
(128, 64, 64) [0 1]
15 110 60 52
(128, 64, 64) [0 1]
16 104 47 46
(128, 64, 64) [0 1]
17 89 57 49
(128, 64, 64) [0 1]
18 62 49 29
(128, 64, 64) [0 1]
19 118 55 54
(128, 64, 64) [0 1]
20 101 55 52
(128, 64, 64) [0 1]
21 90 69 37
(128, 64, 64) [0 1]
22 109 67 56
(128, 64, 64) [0 1]
23 108 58 52
(128, 64, 64) [0 1]
24 101 53 46
(128, 64, 64) [0 1]
25 95 68 43
(128, 64, 64) [0 1]
26 97 58 47
(128, 64, 64) [0 1]
27 77 53 36
(128, 64, 64) [0 1]
28 101 54 45
(128, 64, 64) [0 1]
29 110 59 47
(128, 64, 64) [0 1]
30 90 70 38
(128, 64, 64) [0 1]
31 94 62 46
(128, 64, 64) [0 1]
32 84 70 40
(128, 64, 64) [0 1]
33 110 56 55
(128, 64, 64) [0 1]
34 96 62 45
(128, 64, 64) [0 1]
35 114 59 53
(128, 64, 64) [0 1]
36 106 5

(128, 64, 64) [0 1]
254 89 49 42
(128, 64, 64) [0 1]
255 99 51 55
(128, 64, 64) [0 1]
256 90 51 32
(128, 64, 64) [0 1]
257 114 63 54
(128, 64, 64) [0 1]
258 77 54 40
(128, 64, 64) [0 1]
259 50 35 20
(128, 64, 64) [0 1]
260 80 41 42
(128, 64, 64) [0 1]
261 97 52 48
(128, 64, 64) [0 1]
262 102 56 55
(128, 64, 64) [0 1]
263 107 56 44
(128, 64, 64) [0 1]
264 94 72 55
(128, 64, 64) [0 1]
265 111 66 49
(128, 64, 64) [0 1]
266 100 61 45
(128, 64, 64) [0 1]
267 90 62 46
(128, 64, 64) [0 1]
268 111 57 47
(128, 64, 64) [0 1]
269 105 64 62
(128, 64, 64) [0 1]
270 96 61 43
(128, 64, 64) [0 1]
271 67 58 32
(128, 64, 64) [0 1]
272 77 68 40
(128, 64, 64) [0 1]
273 115 59 60
(128, 64, 64) [0 1]
274 105 64 60
(128, 64, 64) [0 1]
275 95 59 54
(128, 64, 64) [0 1]
276 102 61 45
(128, 64, 64) [0 1]
277 93 64 46
(128, 64, 64) [0 1]
278 94 59 45
(128, 64, 64) [0 1]
279 83 53 42
(128, 64, 64) [0 1]
280 80 60 35
(128, 64, 64) [0 1]
281 117 64 51
(128, 64, 64) [0 1]
282 108 60 54
(128, 64, 64) [0 1]
283 98 60 5

In [6]:
save_name = 'All_bv_128_64_64.pickle'
save_file = open(os.path.join(os.getcwd(),'data',save_name),'wb')
pickle.dump(all_train_data,save_file)
save_file.close()